Hunter S. DiCicco

with Dr. Dongwon Lee, Ph.D.

# Data Collection: Limited Web Scraping
for *Promotional* and *Misreporting* Content

## TODO:
* Rectify Promotional Content/Native Ads Class Balance

### Setup and Imports

In [1]:
import sys
import re
import os
import sys
import requests as r
import itertools as it

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

os.chdir('../')
import feature_extraction as fe

In [2]:
headers = r.utils.default_headers()
headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})

In [3]:
nativead = pd.read_excel("data/Nativead.xlsx")
nativead.head()

,Site,Status,Coder,Story 1,Story 2,Story 3
0,Yahoo News,DONE,DI,https://goingthere.yahoo.com/post/133803688011...,https://goingthere.yahoo.com/post/130087715036...,https://goingthere.yahoo.com/post/130160620511...
1,ABC News \n,Done,DI,http://abcnews.go.com/US/wireStory/providence-...,http://abcnews.go.com/Politics/obama-hits-trum...,http://abcnews.go.com/Politics/justice-antonin...
2,CNN Network,DONE,BWW,http://sponsorcontent.cnn.com/interactive/cine...,http://sponsorcontent.cnn.com/amazon/article/8...,http://sponsorcontent.cnn.com/subaru/article/a...
3,NBC News Digital,DONE,NJE,http://www.nbcnews.com/video/this-designer-cut...,http://www.nbcnews.com/video/baby-boomers-are-...,http://www.nbcnews.com/video/he-creates-furnit...
4,HuffingtonPost.com,DONE,BWW,http://www.huffingtonpost.com/2015/06/04/best-...,http://www.huffingtonpost.com/kathy-magee/a-sm...,http://www.huffingtonpost.com/2015/05/29/evolu...


In [4]:
promo = pd.read_excel("data/PR_content_news.xlsx")
promo.head()

,source,URLs
0,U.S. Department of State,https://www.state.gov/release-of-the-united-st...
1,U.S. Department of State,https://www.state.gov/conclusion-of-negotiatio...
2,U.S. Department of State,https://www.state.gov/u-s-mexico-joint-declara...
3,U.S. Department of State,https://www.state.gov/united-states-restricts-...
4,U.S. Department of State,https://www.state.gov/president-donald-j-trump...


In [5]:
nativead['Status'].value_counts()

DONE                38
Done                 5
N/A, now defunct     1
Found only 1         1
Name: Status, dtype: int64

In [6]:
nativead = nativead.astype(str)
nativead = nativead[nativead['Status'].apply(str.lower)=='done']
nativead.isnull().sum()

Site       0
Status     0
Coder      0
Story 1    0
Story 2    0
Story 3    0
dtype: int64

In [7]:
nativead = nativead.drop(['Status', 'Coder'], axis=1)
nativead.head(n=5)

,Site,Story 1,Story 2,Story 3
0,Yahoo News,https://goingthere.yahoo.com/post/133803688011...,https://goingthere.yahoo.com/post/130087715036...,https://goingthere.yahoo.com/post/130160620511...
1,ABC News \n,http://abcnews.go.com/US/wireStory/providence-...,http://abcnews.go.com/Politics/obama-hits-trum...,http://abcnews.go.com/Politics/justice-antonin...
2,CNN Network,http://sponsorcontent.cnn.com/interactive/cine...,http://sponsorcontent.cnn.com/amazon/article/8...,http://sponsorcontent.cnn.com/subaru/article/a...
3,NBC News Digital,http://www.nbcnews.com/video/this-designer-cut...,http://www.nbcnews.com/video/baby-boomers-are-...,http://www.nbcnews.com/video/he-creates-furnit...
4,HuffingtonPost.com,http://www.huffingtonpost.com/2015/06/04/best-...,http://www.huffingtonpost.com/kathy-magee/a-sm...,http://www.huffingtonpost.com/2015/05/29/evolu...


In [8]:
def get_article_url(urls=pd.DataFrame):
    return np.random.choice(urls.to_numpy().flatten())

story_urls = nativead[['Story 1', 'Story 2', 'Story 3']]
test_url = get_article_url(story_urls)
article = r.get(test_url, headers)
while not article:
    print(test_url + ' Response code 404, trying again...')
    try:
        test_url = get_article_url(story_urls)
        article = r.get(test_url, headers)
    except r.exceptions.MissingSchema:
        continue
else:
    data = article.text
    soup = BeautifulSoup(data)

In [9]:
print("\n".join([para.get_text(strip=True) for para in soup.find_all('p')]))


Partnered content
Presented by Mazda
This is the first article in a four-part special advertising feature series presented by Mazda that looks at the ongoing relationship between technology, innovation and sport.
Technical innovation in sport - it's something we as participants and fans take for granted now. It could be Grand Prix champion Lewis Hamilton talking to his pit crew throughout his thrilling drive in Austin Texas to win the 2015 Formula One title. Or improvements in the safety standards of cricketers' helmets, following the death of Australian Test batsman Phillip Hughes when a delivery struck him at the top of his neck, in Sydney in 2014.
Perhaps the most interesting development is the introduction of ingestible computers that constantly transmit data on a player's vital functions, such as blood pressure and body temperature. Athletes in sports as diverse as motor sport, athletics, football and hockey are already using them. It's the latest innovation in the ever-changing 

In [10]:
sponsored_div = soup.find_all('div', attrs={'class':re.compile("sponsor|provided")})
links = soup.findAll('a', attrs={'href': re.compile("^https*:\/\/")})
#[j.find_all('div', attrs={'class':'item'}) for j in sponsored_div if j.find_all('div', attrs={'class':'item'})!=[]]
[j.get('href') for j in links][0:5]

['https://www.bbc.co.uk',
 'https://www.bbc.co.uk/accessibility/',
 'https://account.bbc.com/account',
 'https://www.bbc.co.uk',
 'https://www.bbc.co.uk/news']

In [11]:
try:
    final_frame = pd.read_csv('data/promotional_text.csv')
except:
    texts = []
    urls = nativead[['Story 1', 'Story 2', 'Story 3']].to_numpy().flatten().tolist() + promo['URLs'].to_list()
    good_urls = []
    more_urls = []
    badresponses = 0
    for i, url in enumerate(urls):
        if url=='nan':
            texts.append('')
            continue
        try:
            article = r.get(url, headers)
            if article:
                good_urls.append(url)
                links = soup.select('body p > a')
                data = article.text
                soup = BeautifulSoup(data)
                article = " ".join([para.get_text(strip=True) for para in soup.find_all('p')])
                texts.append(article)
            else:
                badresponses += 1
                texts.append('')
        except r.exceptions.TooManyRedirects:
            texts.append('')
            continue
        sys.stdout.flush()
        sys.stdout.write("\r" + f"{i!s}/{len(urls)!s} with {badresponses!s} bad responses.")
    sources = nativead['Site'].repeat(3).to_list() + promo['source'].to_list()
    final_frame = pd.DataFrame({'source':sources,
                                'url': urls,
                                'text': texts})
    final_frame = final_frame[final_frame['text']!='']
    final_frame = final_frame[final_frame['text']!=' ']
    final_frame.to_csv('data/promotional_text.csv', index=False)

In [12]:
final_frame.head()

,source,url,text
0,ABC News \r\n,http://abcnews.go.com/Politics/obama-hits-trum..., -- \r\nPresidentObamasaid today that he does...
1,ABC News \r\n,http://abcnews.go.com/Politics/justice-antonin..., -- \r\nEven though the relatives and doctors...
2,CNN Network,http://sponsorcontent.cnn.com/interactive/cine...,“Everything that needs to be said hasalready b...
3,NBC News Digital,http://www.nbcnews.com/video/this-designer-cut...,Watch live: NY Gov. Andrew Cuomo holds coronav...
4,NBC News Digital,http://www.nbcnews.com/video/baby-boomers-are-...,Watch live: NY Gov. Andrew Cuomo holds coronav...


In [13]:
with open('promotional_news_urls.txt', mode='w') as fileout:
    fileout.write("\n".join(final_frame.url.to_list()))

We can now assemble `ArticleVectors` straight-on, supplying one or both of the article text and the URL. This is demonstrated below.

In [14]:
%%timeit
x = fe.ArticleVector(text='sample text', url=test_url)
x.vector

14.3 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
x = fe.ArticleVector(url=test_url)
x.vector

799 ms ± 87.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Full disclosure: Due to limitations in the current version of `goose3` I am unable to properly handle `ReadTimeout` exceptions, which occur when an article takes longer than expected for `requests` to receive data from. I can arbitrarily extend the timeout threshold, but this of course means that each article will take even longer to extract. Instead I opt to use the text that I scraped using `BeautifulSoup` and **omit the article title** and Goose extraction altogether.

This is of no consequence in the end -- the title is only used to determine a portion of an article's `all_caps_index` and `apa_index`.

In [16]:
vectors = [fe.ArticleVector(text=row['text'], url=row['url']) for _, row in final_frame.iterrows()]

In [17]:
with open('data/promotional_news_vectors.txt', mode='w') as fileout:
    fileout.write("\n".join([str(" ".join(list(map(str, j.vector)) + ['9'])) for j in vectors]))

On to the NYTimes BBC Corrections dataset.

I use a Powershell `cmdlet` originally obtained from [this source](http://woshub.com/powershell-get-folder-sizes/) to find those directories whose size is greater than 19.57Kb (the size of an empty Git repository). These then contain text snapshots of the first revisions of articles.

```
gci -force '.' -ErrorAction SilentlyContinue | 
    ? { ($_ -is [io.directoryinfo]) } | % {
        $len = 0
        gci -recurse -force $_.fullname -ErrorAction SilentlyContinue | 
           % { $len += $_.length}
        $_.fullname, ', {0:N2}' -f ($len / 1Mb)
} | Out-File -FilePath './nytbbcdirs.csv' -Encoding UTF8
```

In [18]:
folders = pd.read_csv("data/nytbbcdirs.csv", encoding='utf-8', header=None, names=['dir', 'size'])
folders = folders[folders['size'] > 19.57]

In [19]:
path = folders.max()['dir'].rstrip(" "); path

'C:\\Users\\hunter\\dev\\data\\nytimes_bbc_2017_2020\\2019-05_new'

The folder found above is the most complete archive of the first revisions of corrected articles.

We will [recursively open all bottom-level subfiles](https://stackoverflow.com/a/13617120) as text in the two URL directories. The second line is always article title.

In [20]:
file_set = set()
for dir_, dirs, files in os.walk(path):
    if '.git' in dirs:
        dirs.remove('.git')
    if '.gitignore' in files:
        files.remove('.gitignore')
    for file_name in files:
        rel_dir = os.path.relpath(dir_, path)
        rel_file = os.path.join(rel_dir, file_name)
        file_set.add(rel_file)
file_set = tuple(file_set); file_set[0:5]        

('https_\\www.nytimes.com\\2019\\05\\01\\us\\college-admissions-scandal.html',
 'www.bbc.co.uk\\news\\uk-politics-48360456',
 'www.nytimes.com\\2019\\05\\06\\books\\review\\spring-ali-smith.html',
 'www.nytimes.com\\2019\\05\\07\\business\\carbon-removal-technology-start-ups.html',
 'www.bbc.co.uk\\news\\world-europe-48379620')

In [21]:
titles, texts = [], []

for file in file_set:
    with open(os.path.join(path, file), mode='r', encoding = 'utf-8', errors='ignore') as filein:
        text = filein.readlines()
        texts.append(" ".join(text[2:]))
        titles.append(text[1])

In [22]:
corrections = pd.DataFrame({'title':titles,
                            'text':texts})

In [23]:
feats = ["url_ending_index", "from_reputable_source_index",
         "today_index", "grammar_index", "quotation_index",
         "past_tense_index", "present_tense_index", "should_index",
         "opinion_index", "all_caps_index", "from_satire_source_index",
         "exclamation_index", "apa_index", "name_source_index",
         "interjection_index", "you_index", "dot_gov_ending_index",
         "from_unreputable_source_index", "label"]
try:
    corrections_vectors = pd.read_csv('./data/correction_news_vectors.txt', delim_whitespace=True, header=None, names=feats)
except FileNotFoundError:
    corrections_vectors = []

    for i, row in corrections.iterrows():
        sys.stdout.flush()
        sys.stdout.write("\r" + f"{i!s}/{len(file_set)}")
        corrections_vectors.append(fe.ArticleVector(text=row['text'], title=row['title']).vector)

    corrections_vectors = pd.DataFrame(corrections_vectors, columns=feats)
    corrections_vectors = corrections_vectors.assign(url_ending_index=1)
    corrections_vectors = corrections_vectors.assign(dot_gov_ending_index=0)
    corrections_vectors = corrections_vectors.assign(label=11)

    corrections_vectors.to_csv('./data/correction_news_vectors.txt', header=None, index=None, sep=' ')

In [24]:
corrections_vectors.head()

,url_ending_index,from_reputable_source_index,today_index,grammar_index,quotation_index,past_tense_index,present_tense_index,should_index,opinion_index,all_caps_index,from_satire_source_index,exclamation_index,apa_index,name_source_index,interjection_index,you_index,dot_gov_ending_index,from_unreputable_source_index,label
0,1,0,0.000491,0.074656,0.0,0.052063,0.075147,0.000982,0,0.006876,1,0.000000,1,0.162574,0.000491,0.005403,0,0,11
1,1,0,0.000000,0.061208,0.0,0.099256,0.064516,0.001654,0,0.008271,1,0.000000,0,0.111663,0.000827,0.004136,0,0,11
2,1,0,0.001984,0.063492,0.0,0.041667,0.069444,0.000000,0,0.037698,1,0.000000,0,0.253968,0.000000,0.007937,0,0,11
3,1,0,0.000000,0.040084,0.0,0.061181,0.072785,0.000000,0,0.004219,1,0.001055,1,0.113924,0.001055,0.003165,0,0,11
4,1,0,0.005063,0.035443,0.0,0.093671,0.043038,0.000000,0,0.007595,1,0.000000,0,0.230380,0.000000,0.000000,0,0,11
